<a href="https://colab.research.google.com/github/YangyangFu/transformer-time-series/blob/main/examples/train_informer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Install Dependency

In [1]:
!pip install git+https://github.com/YangyangFu/transformer-time-series@main

  Cloning https://github.com/YangyangFu/transformer-time-series (to revision main) to /tmp/pip-req-build-1kjt68wh
  Running command git clone --filter=blob:none --quiet https://github.com/YangyangFu/transformer-time-series /tmp/pip-req-build-1kjt68wh
  Resolved https://github.com/YangyangFu/transformer-time-series to commit 94dff8ea3113b4dcb7081867dc655d43351f57d9
  Preparing metadata (setup.py) ... done
  Created wheel for tsl: filename=tsl-1.0-py3-none-any.whl size=41953 sha256=e2578e9b860917d8c1faf40022b8e89c4207e70bebd612adebc0b0d63a7629f2
  Stored in directory: /tmp/pip-ephem-wheel-cache-8j0oma6o/wheels/1e/96/45/2b21250d4fcf59625a099e8fb90e3b43278918abd7b7747e49
Successfully built tsl


In [2]:
import tensorflow as tf
import os
import random
import numpy as np

from tsl.transformers.informer import DataLoader
from tsl.transformers.informer import Informer


def set_seed(seed: int = 42) -> None:
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    tf.experimental.numpy.random.seed(seed)
    tf.random.set_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

set_seed(42)


Random seed set as 42


# Download Data

In [3]:
!apt install subversion
!svn checkout https://github.com/YangyangFu/transformer-time-series/trunk/datasets
!bash ./datasets/download_data.sh

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libapr1 libaprutil1 libserf-1-1 libsvn1 libutf8proc2
Suggested packages:
  db5.3-util libapache2-mod-svn subversion-tools
The following NEW packages will be installed:
  libapr1 libaprutil1 libserf-1-1 libsvn1 libutf8proc2 subversion
0 upgraded, 6 newly installed, 0 to remove and 16 not upgraded.
Need to get 2,672 kB of archives.
After this operation, 10.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libapr1 amd64 1.7.0-8ubuntu0.22.04.1 [108 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libaprutil1 amd64 1.6.1-5ubuntu4.22.04.1 [92.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libserf-1-1 amd64 1.3.9-10ubuntu2 [50.0 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libutf8proc2 amd64 2.7.0-3 [73.9 kB]
Get:5 http://archive.

# Experiment Settings


In [4]:
embed_dim = 512
source_seq_len = 64
target_seq_len = 128
pred_len = 96
n_num_covs = 7
n_targets = 1

MAX_EPOCHS = 10

# Create Data Loader

In [ ]:
# get data path
data_path = "./ETT-small/ETTh1.csv"

# create dataloader
dataloader = DataLoader(data_path=data_path,
                    target_cols=['OT'],
                    num_cov_cols=['HUFL','HULL','MUFL','MULL','LUFL','LULL','OT'],
                    train_range=(0, 24*30*12),
                    val_range=(24*30*12, 24*20*16),
                    test_range=(24*30*16, 24*30*20),
                    hist_len=source_seq_len,
                    token_len=target_seq_len-pred_len,
                    pred_len=pred_len,
                    batch_size=32,
                    use_time_features=True
                    )
train_ds = dataloader.generate_dataset(mode="train", shuffle=True, seed=1)
val_ds = dataloader.generate_dataset(mode="validation", shuffle=False, seed=1)
test_ds = dataloader.generate_dataset(mode="test", shuffle=False, seed=1)



Generating time features.................


 61%|██████    | 10597/17420 [03:52<02:59, 37.92it/s]

In [17]:
# create informer model
model = Informer(output_dim=n_targets,
                pred_len=pred_len,
                num_layers_encoder=4,
                num_heads_encoder=16,
                key_dim_encoder=32,
                value_dim_encoder=32,
                output_dim_encoder=512,
                hidden_dim_encoder=2048,
                factor_encoder=4,
                num_layers_decoder=2,
                num_heads_decoder=8,
                key_dim_decoder=64,
                value_dim_decoder=64,
                output_dim_decoder=512,
                hidden_dim_decoder=2048,
                factor_decoder=4,
                num_cat_cov=0,
                cat_cov_embedding_size=[],
                cat_cov_embedding_dim=16,
                freq='H',
                use_holiday=True,
                dropout_rate=0.1,)

# training settings
loss_fn = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

train_metrics = [tf.keras.metrics.MeanAbsoluteError()]
val_metrics = [tf.keras.metrics.MeanAbsoluteError()]

# train step
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        x_enc, x_dec = x
        y_pred = model(x_enc, x_dec, training=True)
        loss = loss_fn(y, y_pred)

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    # update metrics
    for metric in train_metrics:
            metric.update_state(y, y_pred)
    return loss

# validation step
@tf.function
def val_step(x, y):
    x_enc, x_dec = x
    y_pred = model(x_enc, x_dec, training=False)
    loss = loss_fn(y, y_pred)
    for metric in val_metrics:
        metric.update_state(y, y_pred)
    return loss

In [ ]:
# main loop
for epoch in range(MAX_EPOCHS):
    # take a batch
    for batch in train_ds:
        num_covs, cat_covs, time_enc, time_dec, target_dec = batch

        try:
            # zero for target
            token_dec = target_dec[:, :-pred_len, :]
            zeros = tf.zeros_like(target_dec[:, -pred_len:, :])
            token_target_dec = tf.concat([token_dec, zeros], axis=1)

            # feed model
            x_enc = (num_covs, cat_covs, time_enc)
            x_dec = (time_dec, token_target_dec)

            # train step
            loss = train_step((x_enc, x_dec), target_dec[:, -pred_len:, :])

        except tf.errors.OutOfRangeError:
            pass

    # print loss every epoch
    print(f"Epoch {epoch+1}/{MAX_EPOCHS} training loss: {loss:.4f}, MAE: {train_metrics[0].result():.4f}")

    # reset train metrics
    for metric in train_metrics:
        metric.reset_states()

    # run validation loop
    # how to run validaiton loop without batching?

    for val_batch in val_ds:
        num_covs, cat_covs, time_enc, time_dec, target_dec = val_batch

        try:
            # zero for target
            token_dec = target_dec[:, :-pred_len, :]
            zeros = tf.zeros_like(target_dec[:, -pred_len:, :])
            token_target_dec = tf.concat([token_dec, zeros], axis=1)

            # feed model
            x_enc = (num_covs, cat_covs, time_enc)
            x_dec = (time_dec, token_target_dec)

            # calculate loss
            loss_val = val_step((x_enc, x_dec), target_dec[:, -pred_len:, :])

        except tf.errors.OutOfRangeError:
            pass

        # print loss every epoch
    print(f"Epoch {epoch+1}/{MAX_EPOCHS} validation loss: {loss_val:.4f}, MAE: {val_metrics[0].result():.4f}")

    # reset val metrics
    for metric in val_metrics:
        metric.reset_states()